# Panda's Advanced Dataframes Exercises
<hr style="border:2px solid red"> </hr>


In [1]:
import pandas as pd
import numpy as np

In [2]:
#this will be what we use every time we want to talk to a database in SQL
def get_db_url(database_name):
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{database_name}'

In [3]:
sql_query_example = """SELECT * from departments"""

In [4]:
database = get_db_url("employees") #DO NOT PRINT THIS! It will show your password!

In [5]:
df = pd.read_sql(sql_query_example , database)
df.head()

,dept_no,dept_name
0,d009,Customer Service
1,d005,Development
2,d002,Finance
3,d003,Human Resources
4,d001,Marketing


### 5. Once you have successfully run a query:
#### a. Intentionally make a typo in the database url. What kind of error message do you see?



In [6]:
error_database = get_db_url("duckduckgoose") #DO NOT PRINT THIS! It will show your password!

#### b. Intentionally make an error in your SQL query. What does the error message look like?

In [66]:
sql_query_error = """ SELECT ** FROM employees"""
df = pd.read_sql(sql_query_error , database)
df.head()

ProgrammingError: (pymysql.err.ProgrammingError) (1064, "You have an error in your SQL syntax; check the manual that corresponds to your MySQL server version for the right syntax to use near '* FROM employees' at line 1")
[SQL:  SELECT ** FROM employees]
(Background on this error at: http://sqlalche.me/e/14/f405)

### 6. Read the employees and titles tables into two separate DataFrames.



In [10]:
employees_table = pd.read_sql("""SELECT * FROM employees""" , get_db_url("employees"))
titles_table = pd.read_sql("""SELECT * FROM titles""" , get_db_url("employees"))

### 7. How many rows and columns do you have in each DataFrame? Is that what you expected?

In [11]:
employees_table.shape 

(300024, 6)

In [12]:
titles_table.shape #yes, there are some employees whom have held more than one title while working for this company, so there are more title entries than employees

(443308, 4)

### 8. Display the summary statistics for each DataFrame.



In [67]:
employees_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300024 entries, 0 to 300023
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   emp_no      300024 non-null  int64 
 1   birth_date  300024 non-null  object
 2   first_name  300024 non-null  object
 3   last_name   300024 non-null  object
 4   gender      300024 non-null  object
 5   hire_date   300024 non-null  object
dtypes: int64(1), object(5)
memory usage: 13.7+ MB


In [68]:
titles_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 443308 entries, 0 to 443307
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   emp_no     443308 non-null  int64 
 1   title      443308 non-null  object
 2   from_date  443308 non-null  object
 3   to_date    443308 non-null  object
dtypes: int64(1), object(3)
memory usage: 13.5+ MB


### 9. How many unique titles are in the `titles` DataFrame?



In [21]:
#python
titles_table.title.unique().size

7

In [22]:
#using a SQL query
unique_titles = pd.read_sql("""SELECT DISTINCT title FROM titles""" , get_db_url("employees"))
unique_titles

,title
0,Senior Engineer
1,Staff
2,Engineer
3,Senior Staff
4,Assistant Engineer
5,Technique Leader
6,Manager


### 10. What is the oldest date in the to_date column?



In [27]:
#oldest is being interpreted as the date furthest back from today
#python
titles_table.to_date.min()

datetime.date(1985, 3, 1)

In [28]:
#using a SQL query
oldest_to_date = pd.read_sql("""SELECT *
                                FROM titles
                                ORDER BY to_date
                                LIMIT 1""" ,
                                get_db_url("employees"))
oldest_to_date

,emp_no,title,from_date,to_date
0,20869,Engineer,1985-02-17,1985-03-01


### 11. What is the most recent date in the to_date column?



In [61]:
test_date = titles_table.to_date.head(1)
[test_date == titles_table.to_date.max()]

[0    True
 Name: to_date, dtype: bool]

In [70]:
titles_table.to_date.max()

datetime.date(9999, 1, 1)

In [65]:
#most recent is being interpreted as the date closest to today without going past

#python
titles_table[titles_table.to_date != titles_table.to_date.max()].to_date.max()


datetime.date(2002, 8, 1)

In [30]:
#using a SQL query
recent_to_date = pd.read_sql("""SELECT *
                                FROM titles
                                WHERE year(to_date) != 9999 
                                ORDER BY to_date DESC""" ,
                                get_db_url("employees"))
recent_to_date.head()

,emp_no,title,from_date,to_date
0,495525,Staff,1996-08-01,2002-08-01
1,446732,Senior Engineer,1992-11-10,2002-08-01
2,447596,Staff,1996-08-01,2002-08-01
3,448129,Senior Staff,2001-01-23,2002-08-01
4,452819,Engineer,1993-08-01,2002-08-01
